# Tutorial 5
This tutorial is to demonstrate an example of using ETEO for order execution

## Step 1: Import Packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
from pathlib import Path
import os
import torch

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.transition.builder import build_transition
from collections import Counter
from trademaster.utils import set_seed
set_seed(2023)

2023-03-04 18:50:55,112	INFO services.py:1476 -- View the Ray dashboard at http://127.0.0.1:8269
2023-03-04 18:50:57,245	INFO worker.py:973 -- Calling ray.init() again after it has already been called.


## Step 2: Import Configs

In [2]:
parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "order_execution",
                                                 "order_execution_BTC_eteo_eteo_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
parser.add_argument("--test_style", type=str, default="-1")
args,_ = parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)

## Step 3: Build Dataset

In [3]:
dataset = build_dataset(cfg)

The 18_th segment length is less than the min length so it won't be tested


## Step 4: Build Environment

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="train"))
valid_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="valid"))
test_environment = build_environment(cfg, default_args=dict(dataset=dataset, task="test"))

In [5]:
train_environment.df.head()

,system_time,midpoint,spread,buys,sells,bids_distance_0,bids_distance_1,bids_distance_2,bids_distance_3,bids_distance_4,...,asks_market_notional_5,asks_market_notional_6,asks_market_notional_7,asks_market_notional_8,asks_market_notional_9,asks_market_notional_10,asks_market_notional_11,asks_market_notional_12,asks_market_notional_13,asks_market_notional_14
0,2021-04-07 11:33:41.122161+00:00,55896.285,0.01,4.448599e+06,3.899402e+06,-8.945138e-08,-0.000065,-0.000073,-0.000154,-0.000162,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1176.72998,0.0
1,2021-04-07 11:34:41.122161+00:00,55948.685,1.43,1.243244e+06,3.606502e+06,-1.277957e-05,-0.000013,-0.000242,-0.000242,-0.000259,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
2,2021-04-07 11:35:41.122161+00:00,56013.785,0.01,3.069094e+06,1.575759e+06,-8.926374e-08,-0.000009,-0.000070,-0.000088,-0.000173,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
3,2021-04-07 11:36:41.122161+00:00,55903.575,7.17,1.220819e+06,1.321487e+06,-6.412827e-05,-0.000182,-0.000188,-0.000225,-0.000310,...,0.0,312.040009,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
4,2021-04-07 11:37:41.122161+00:00,55899.995,0.01,2.011287e+06,3.083005e+06,-8.944544e-08,-0.000129,-0.000130,-0.000143,-0.000201,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


## Step 5: Build Net

In [6]:
action_dim = train_environment.action_dim
state_dim = train_environment.state_dim

cfg.act.update(dict(action_dim=action_dim, state_dim=state_dim))
cfg.cri.update(dict(action_dim=action_dim, state_dim=state_dim))

act = build_net(cfg.act)
cri = build_net(cfg.cri)

## Step 6: Build Optimizer

In [7]:
act_optimizer = build_optimizer(cfg, default_args=dict(params=act.parameters()))
cri_optimizer = None

## Step 7: Build Loss

In [8]:
criterion = build_loss(cfg)

## Step 8: Build Transition

In [9]:
transition = build_transition(cfg)

## Step 9: Build Agent

In [10]:
agent = build_agent(cfg, default_args=dict(action_dim=action_dim,state_dim=state_dim,act=act,cri=cri,act_optimizer=act_optimizer,cri_optimizer=cri_optimizer,criterion=criterion,transition=transition,device=device))

## Step 10: Build Trainer

In [11]:
trainer = build_trainer(cfg, default_args=dict(train_environment=train_environment,valid_environment=valid_environment,test_environment=test_environment,agent=agent,device=device))

| Arguments Keep work_dir: /data/sunshuo/qml/TradeMaster/work_dir/order_execution_BTC_eteo_eteo_adam_mse


## Step 11: RL Agent Training
Train the ETTO agent based on the config and save results in workdir

In [12]:
trainer.train_and_valid()

Train Episode: [1/2]
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 37461.153500 | 41059.122004 |    -8.762897%   |
+--------------+--------------+-----------------+
Valid Episode: [1/2]
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 38610.495450 | 37244.542270 |    3.667526%    |
+--------------+--------------+-----------------+
Train Episode: [2/2]
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 37461.153500 | 41059.122004 |    -8.762897%   |
+--------------+--------------+-----------------+
Valid Episode: [2/2]
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 38610.495450 |

## Step 12: RL Agent Testing

In [13]:
trainer.test();


Resume checkpoint /data/sunshuo/qml/TradeMaster/work_dir/order_execution_BTC_eteo_eteo_adam_mse/checkpoints/best.pth
Test Best Episode
+--------------+--------------+-----------------+
|  Cash Left   |     TWAP     | Cash Left Ratio |
+--------------+--------------+-----------------+
| 44920.363750 | 39814.366548 |    12.824509%   |
+--------------+--------------+-----------------+
